In [6]:
import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, Set, Param, NonNegativeReals, Binary, SolverFactory, minimize, value

In [7]:
demand_df = pd.read_csv("Data/dummy_demand.csv")
supply_df = pd.read_csv("Data/dummy_supply.csv")
costs_df = pd.read_csv("Data/dummy_costs.csv")

In [ ]:
# Define model
model = ConcreteModel()

# Sets
model.S = Set(initialize=supply_df['Supply Site'].unique())
model.D = Set(initialize=demand_df['Demand Site'].unique())
model.T = Set(initialize=demand_df['Year'].unique())

# Define parameters
setup_costs = supply_df.set_index('Supply Site')['Setup Costs'].to_dict()
model.SetupCost = Param(model.S, initialize=setup_costs)

demand_data = demand_df.set_index(['Demand Site', 'Year'])['Demand'].to_dict()
model.Demand = Param(model.D, model.T, initialize=demand_data)

# Energy supply costs
supply_costs = costs_df.set_index(['Supply Site', 'Demand Site', 'Year'])['Supply Cost'].to_dict()
model.SupplyCost = Param(model.S, model.D, model.T, initialize=supply_costs)

#Maximum Capacity parameter
max_capacity = supply_df.set_index(['Supply Site'])['Capacity'].to_dict()
model.MaxCapacity = Param(model.S, initialize=max_capacity)

# Decision variables
model.y = Var(model.S, domain=Binary) # y determines whether a supply site is active
model.x = Var(model.S, model.D, model.T, domain=NonNegativeReals) # x determines amount supplied at time t by one site to another

def objective_rule(model):
    setup_cost = sum(model.SetupCost[s]*model.y[s] for s in model.S)
    supply_cost = sum(model.SupplyCost[s, d, t] * model.x[s, d, t]
                      for s in model.S for d in model.D for t in model.T)
    return setup_cost + supply_cost

model.Objective = Objective(rule=objective_rule, sense=minimize)

# Constraints

# demand satisfaction
def demand_satisfaction_rule(model, d, t):
    return sum(model.x[s, d, t] for s in model.S) >= model.Demand[d, t]
model.DemandSatisfaction = Constraint(model.D, model.T, rule=demand_satisfaction_rule)

# Supply capacity
def supply_capacity_rule(model, s):
    return sum(model.x[s, d, t] for d in model.D for t in model.T) <= model.y[s] * model.MaxCapacity[s]
model.SupplyCapacity = Constraint(model.S, rule=supply_capacity_rule)

# Solve the model
solver = SolverFactory('gurobi')
results = solver.solve(model)

print("Optimal Solution: ")
for s in model.S:
    if model.y[s].value > 0.5:
        print(f"Supply Site {s} is active")
        for d in model.D:
            for t in model.T:
                if model.x[s, d, t].value > 0:
                    print(f" Supplied {model.x[s, d, t].value:.2f} to {d} in year {t}")

supply_results = []
for s in model.S:
    for d in model.D:
        for t in model.T:
            energy_provided = value(model.x[s, d, t])
            if energy_provided > 0:
                supply_results.append({
                    'Supply Site': s,
                    'Demand Site': d,
                    'Year': t,
                    'Energy Provided (MW)': energy_provided
                })
supply_results_df = pd.DataFrame(supply_results)

constructed_sites = [s for s in model.S if model.y[s].value == 1]
final_cost = model.obj.expr
solve_time = results.solver.time
summary_df = pd.DataFrame({
    "Construct Sites": [constructed_sites],
    "Final Cost": [final_cost],
    "Solve Time (seconds)": [solve_time]
})

summary_df

                    

Optimal Solution: 
Supply Site Glendoe Hydro Scheme is active
 Supplied 100.00 to City of London in year 2005
 Supplied 100.00 to City of London in year 2006
 Supplied 100.00 to City of London in year 2007
 Supplied 100.00 to City of London in year 2008
 Supplied 100.00 to City of London in year 2009
 Supplied 100.00 to City of London in year 2010
Supply Site Gate Burton - Solar & Energy Storage Park is active
 Supplied 100.00 to Birmingham in year 2005
 Supplied 100.00 to Birmingham in year 2006
 Supplied 100.00 to Birmingham in year 2007
 Supplied 100.00 to Birmingham in year 2008
 Supplied 100.00 to Birmingham in year 2009
 Supplied 100.00 to Birmingham in year 2010
Supply Site Berwick Bank Offshore Wind Farm  is active
 Supplied 100.00 to City of Edinburgh in year 2005
 Supplied 100.00 to City of Edinburgh in year 2006
 Supplied 100.00 to City of Edinburgh in year 2007
 Supplied 100.00 to City of Edinburgh in year 2008
 Supplied 100.00 to City of Edinburgh in year 2009
 Supplied 10

AttributeError: 'ConcreteModel' object has no attribute 'obj'

In [14]:
def setup_model(demand_df, supply_df, costs_df):

    # Define model
    model = ConcreteModel()

    # Sets
    model.S = Set(initialize=supply_df['Supply Site'].unique())
    model.D = Set(initialize=demand_df['Demand Site'].unique())
    model.T = Set(initialize=demand_df['Year'].unique())

    # Define parameters
    setup_costs = supply_df.set_index('Supply Site')['Setup Costs'].to_dict()
    model.SetupCost = Param(model.S, initialize=setup_costs)

    demand_data = demand_df.set_index(['Demand Site', 'Year'])['Demand'].to_dict()
    model.Demand = Param(model.D, model.T, initialize=demand_data)

    # Energy supply costs
    supply_costs = costs_df.set_index(['Supply Site', 'Demand Site', 'Year'])['Supply Cost'].to_dict()
    model.SupplyCost = Param(model.S, model.D, model.T, initialize=supply_costs)

    #Maximum Capacity parameter
    max_capacity = supply_df.set_index(['Supply Site'])['Capacity'].to_dict()
    model.MaxCapacity = Param(model.S, initialize=max_capacity)

    # Decision variables
    model.y = Var(model.S, domain=Binary) # y determines whether a supply site is active
    model.x = Var(model.S, model.D, model.T, domain=NonNegativeReals) # x determines amount supplied at time t by one site to another

    def objective_rule(model):
        setup_cost = sum(model.SetupCost[s]*model.y[s] for s in model.S)
        supply_cost = sum(model.SupplyCost[s, d, t] * model.x[s, d, t]
                        for s in model.S for d in model.D for t in model.T)
        return setup_cost + supply_cost

    model.Objective = Objective(rule=objective_rule, sense=minimize)

    # Constraints

    # demand satisfaction
    def demand_satisfaction_rule(model, d, t):
        return sum(model.x[s, d, t] for s in model.S) >= model.Demand[d, t]
    model.DemandSatisfaction = Constraint(model.D, model.T, rule=demand_satisfaction_rule)

    # Supply capacity
    def supply_capacity_rule(model, s, t):
        return sum(model.x[s, d, t] for d in model.D) <= model.y[s] * model.MaxCapacity[s]
    model.SupplyCapacity = Constraint(model.S, model.T, rule=supply_capacity_rule)

    return model

def create_results_dfs(results, model):

    supply_results = []
    for s in model.S:
        for d in model.D:
            for t in model.T:
                energy_provided = value(model.x[s, d, t])
                if energy_provided > 0:
                    supply_results.append({
                    'Supply Site': s,
                    'Demand Site': d,
                    'Year': t,
                    'Energy Provided (MW)': energy_provided
                    })
    supply_results_df = pd.DataFrame(supply_results)

    constructed_sites = [s for s in model.S if model.y[s].value == 1]
    solve_time = results.solver.time
    summary_df = pd.DataFrame({
        "Constructed Sites": [constructed_sites],
        "Solve Time (seconds)": [solve_time],
        "Final Objective Value": [value(model.Objective)]
    })

    return supply_results_df, summary_df


def run_model(demand_df, supply_df, costs_df, output_directory="", output_suffix="", model_solver='gurobi',
              save_results=True):
    
    #create the model
    model = setup_model(demand_df, supply_df, costs_df)

    #create the solver
    solver = SolverFactory(model_solver)
    results = solver.solve(model)

    supply_results, summary_results = create_results_dfs(results, model)
    
    if save_results:
        supply_results.to_csv(f"{output_directory}supply_results{output_suffix}.csv",
                               index=False)
        summary_results.to_csv(f"{output_directory}summary_results{output_suffix}.csv",
                               index=False)
    return supply_results, summary_results




In [15]:
run_model(demand_df=demand_df, supply_df=supply_df, costs_df=costs_df, 
          output_directory="Outputs/", output_suffix='_base_test_1')

(                                  Supply Site        Demand Site  Year  \
 0                        Glendoe Hydro Scheme     City of London  2005   
 1                        Glendoe Hydro Scheme     City of London  2006   
 2                        Glendoe Hydro Scheme     City of London  2007   
 3                        Glendoe Hydro Scheme     City of London  2008   
 4                        Glendoe Hydro Scheme     City of London  2009   
 5                        Glendoe Hydro Scheme     City of London  2010   
 6   Gate Burton - Solar & Energy Storage Park         Birmingham  2005   
 7   Gate Burton - Solar & Energy Storage Park         Birmingham  2006   
 8   Gate Burton - Solar & Energy Storage Park         Birmingham  2007   
 9   Gate Burton - Solar & Energy Storage Park         Birmingham  2008   
 10  Gate Burton - Solar & Energy Storage Park         Birmingham  2009   
 11  Gate Burton - Solar & Energy Storage Park         Birmingham  2010   
 12           Berwick Ban

In [16]:
costs_df_2 = pd.read_csv("Data/dummy_costs_2.csv")
run_model(demand_df=demand_df, supply_df=supply_df, costs_df=costs_df_2, 
          output_directory="Outputs/", output_suffix='_base_test_2')

(                                  Supply Site        Demand Site  Year  \
 0                        Glendoe Hydro Scheme     City of London  2005   
 1                        Glendoe Hydro Scheme     City of London  2006   
 2                        Glendoe Hydro Scheme     City of London  2007   
 3                        Glendoe Hydro Scheme     City of London  2008   
 4                        Glendoe Hydro Scheme     City of London  2009   
 5                        Glendoe Hydro Scheme     City of London  2010   
 6                        Glendoe Hydro Scheme  City of Edinburgh  2010   
 7                        Glendoe Hydro Scheme         Birmingham  2010   
 8   Gate Burton - Solar & Energy Storage Park         Birmingham  2005   
 9   Gate Burton - Solar & Energy Storage Park         Birmingham  2006   
 10  Gate Burton - Solar & Energy Storage Park         Birmingham  2007   
 11  Gate Burton - Solar & Energy Storage Park         Birmingham  2008   
 12  Gate Burton - Solar 